## Introduzione

Questa lezione tratterà:
- Che cos'è la chiamata di funzione e i suoi casi d'uso
- Come creare una chiamata di funzione utilizzando Azure OpenAI
- Come integrare una chiamata di funzione in un'applicazione

## Obiettivi di apprendimento

Al termine di questa lezione saprai come e capirai:

- Lo scopo dell'utilizzo della chiamata di funzione
- Configurare la chiamata di funzione utilizzando il servizio Azure OpenAI
- Progettare chiamate di funzione efficaci per il caso d'uso della tua applicazione


## Comprendere le chiamate di funzione

In questa lezione, vogliamo sviluppare una funzionalità per la nostra startup educativa che permetta agli utenti di utilizzare un chatbot per trovare corsi tecnici. Consiglieremo corsi adatti al loro livello di competenza, ruolo attuale e tecnologia di interesse.

Per realizzare questo obiettivo useremo una combinazione di:
 - `Azure Open AI` per creare un'esperienza di chat per l'utente
 - `Microsoft Learn Catalog API` per aiutare gli utenti a trovare corsi in base alla loro richiesta
 - `Function Calling` per prendere la richiesta dell'utente e inviarla a una funzione che effettuerà la chiamata API.

Per iniziare, vediamo perché potremmo voler utilizzare le chiamate di funzione:

print("Messaggi nella prossima richiesta:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # ottieni una nuova risposta da GPT che può vedere la risposta della funzione


print(second_response.choices[0].message)


### Perché usare Function Calling

Se hai già seguito altre lezioni di questo corso, probabilmente conosci la potenza dei Large Language Models (LLM). Speriamo che tu abbia anche notato alcune delle loro limitazioni.

Function Calling è una funzionalità dell’Azure Open AI Service pensata per superare queste limitazioni:
1) Formato di risposta coerente
2) Possibilità di utilizzare dati provenienti da altre fonti di un’applicazione all’interno di una chat

Prima di function calling, le risposte di un LLM erano spesso non strutturate e incoerenti. Gli sviluppatori dovevano scrivere codice di validazione complesso per gestire tutte le possibili variazioni delle risposte.

Gli utenti non potevano ricevere risposte come “Qual è il meteo attuale a Stoccolma?”. Questo perché i modelli erano limitati ai dati disponibili al momento dell’addestramento.

Vediamo l’esempio qui sotto che illustra questo problema:

Supponiamo di voler creare un database di dati degli studenti per suggerire loro il corso più adatto. Qui sotto ci sono due descrizioni di studenti che sono molto simili nei dati che contengono.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vogliamo inviare questi dati a un LLM per analizzarli. In seguito, potremo usarli nella nostra applicazione per inviarli a un'API o salvarli in un database.

Creiamo due prompt identici con cui indichiamo all'LLM quali informazioni ci interessano:


Vogliamo inviare questo a un LLM per analizzare le parti che sono importanti per il nostro prodotto. Così possiamo creare due prompt identici per istruire l'LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Dopo aver creato questi due prompt, li invieremo al LLM utilizzando `openai.ChatCompletion`. Conserviamo il prompt nella variabile `messages` e assegniamo il ruolo a `user`. Questo serve a simulare un messaggio di un utente scritto a un chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Anche se i prompt sono gli stessi e le descrizioni sono simili, possiamo ottenere formati diversi per la proprietà `Grades`.

Se esegui la cella sopra più volte, il formato può essere `3.7` oppure `3.7 GPA`.

Questo succede perché il LLM riceve dati non strutturati sotto forma di prompt scritto e restituisce anch'esso dati non strutturati. Abbiamo bisogno di un formato strutturato per sapere cosa aspettarci quando memorizziamo o utilizziamo questi dati.

Utilizzando la chiamata funzionale, possiamo assicurarci di ricevere dati strutturati. Quando si usa la chiamata di funzione, il LLM in realtà non chiama né esegue alcuna funzione. Invece, creiamo una struttura che il LLM deve seguire nelle sue risposte. Poi usiamo queste risposte strutturate per sapere quale funzione eseguire nelle nostre applicazioni.


![Diagramma di flusso della chiamata di funzione](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.it.png)


### Casi d’uso per l’utilizzo delle chiamate di funzione

**Chiamare strumenti esterni**  
I chatbot sono ottimi nel fornire risposte alle domande degli utenti. Utilizzando le chiamate di funzione, i chatbot possono usare i messaggi degli utenti per svolgere determinati compiti. Ad esempio, uno studente può chiedere al chatbot di "Inviare un’email al mio docente dicendo che ho bisogno di ulteriore assistenza su questa materia". Questo può generare una chiamata alla funzione `send_email(to: string, body: string)`

**Creare richieste API o query al database**  
Gli utenti possono trovare informazioni usando il linguaggio naturale che viene poi convertito in una query formattata o in una richiesta API. Un esempio potrebbe essere un insegnante che chiede "Chi sono gli studenti che hanno completato l’ultimo compito", che potrebbe chiamare una funzione chiamata `get_completed(student_name: string, assignment: int, current_status: string)`

**Creazione di dati strutturati**  
Gli utenti possono prendere un blocco di testo o un file CSV e usare il LLM per estrarre le informazioni più importanti. Ad esempio, uno studente può convertire un articolo di Wikipedia sugli accordi di pace per creare flash card AI. Questo può essere fatto utilizzando una funzione chiamata `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Creare la tua prima chiamata di funzione

Il processo per creare una chiamata di funzione include 3 passaggi principali:
1. Chiamare l’API Chat Completions con un elenco delle tue funzioni e un messaggio dell’utente
2. Leggere la risposta del modello per eseguire un’azione, ad esempio eseguire una funzione o una chiamata API
3. Effettuare un’altra chiamata all’API Chat Completions con la risposta della tua funzione per utilizzare queste informazioni e creare una risposta per l’utente.


![Flusso di una chiamata di funzione](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.it.png)


### Elementi di una chiamata di funzione

#### Input dell’utente

Il primo passo è creare un messaggio utente. Questo può essere assegnato dinamicamente prendendo il valore da un campo di testo, oppure puoi assegnare un valore direttamente qui. Se è la prima volta che lavori con l’API Chat Completions, dobbiamo definire il `role` e il `content` del messaggio.

Il `role` può essere `system` (per creare regole), `assistant` (il modello) oppure `user` (l’utente finale). Per la chiamata di funzione, lo imposteremo su `user` e forniremo una domanda di esempio.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Creazione delle funzioni.

Ora definiremo una funzione e i suoi parametri. Useremo solo una funzione qui chiamata `search_courses`, ma puoi crearne diverse.

**Importante** : Le funzioni vengono incluse nel messaggio di sistema inviato al LLM e verranno conteggiate nel numero di token disponibili.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definizioni**

`name` - Il nome della funzione che vogliamo venga chiamata.

`description` - Questa è la descrizione di come funziona la funzione. Qui è importante essere specifici e chiari.

`parameters` - Un elenco di valori e il formato che si desidera che il modello produca nella sua risposta.

`type` - Il tipo di dato in cui verranno memorizzate le proprietà.

`properties` - Elenco dei valori specifici che il modello utilizzerà per la sua risposta.

`name` - Il nome della proprietà che il modello userà nella sua risposta formattata.

`type` - Il tipo di dato di questa proprietà.

`description` - Descrizione della proprietà specifica.

**Opzionale**

`required` - Proprietà richiesta affinché la chiamata della funzione venga completata.


### Effettuare la chiamata alla funzione
Dopo aver definito una funzione, ora dobbiamo includerla nella chiamata all’API Chat Completion. Lo facciamo aggiungendo `functions` alla richiesta. In questo caso `functions=functions`.

C’è anche la possibilità di impostare `function_call` su `auto`. Questo significa che lasciamo decidere al LLM quale funzione chiamare in base al messaggio dell’utente, invece di assegnarla noi direttamente.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Ora diamo un’occhiata alla risposta e vediamo come è formattata:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Puoi vedere che viene chiamato il nome della funzione e, dal messaggio dell’utente, il LLM è stato in grado di trovare i dati da inserire negli argomenti della funzione.


## 3. Integrare le chiamate di funzione in un'applicazione.

Dopo aver testato la risposta formattata dal LLM, ora possiamo integrarla in un'applicazione.

### Gestire il flusso

Per integrare questo nel nostro applicativo, seguiamo questi passaggi:

Per prima cosa, effettuiamo la chiamata ai servizi Open AI e salviamo il messaggio in una variabile chiamata `response_message`.


In [ ]:
response_message = response.choices[0].message

Ora definiremo la funzione che chiamerà l'API di Microsoft Learn per ottenere un elenco di corsi:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Come buona pratica, vedremo poi se il modello desidera chiamare una funzione. Successivamente, creeremo una delle funzioni disponibili e la abbineremo alla funzione che viene chiamata.
Poi prenderemo gli argomenti della funzione e li mapperemo agli argomenti provenienti dal LLM.

Infine, aggiungeremo il messaggio di chiamata della funzione e i valori che sono stati restituiti dal messaggio `search_courses`. Questo fornisce al LLM tutte le informazioni necessarie per
rispondere all’utente utilizzando il linguaggio naturale.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Sfida di Codice

Ottimo lavoro! Per continuare ad approfondire l’uso di Azure Open AI Function Calling puoi realizzare: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Altri parametri della funzione che potrebbero aiutare chi sta imparando a trovare più corsi. Puoi trovare i parametri disponibili dell’API qui:
 - Crea un’altra chiamata di funzione che raccolga più informazioni dall’utente, come la sua lingua madre
 - Gestisci gli errori quando la chiamata di funzione e/o la chiamata API non restituisce corsi adatti



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni di carattere critico, si raccomanda una traduzione professionale effettuata da un essere umano. Non siamo responsabili per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
